<a href="https://colab.research.google.com/github/rajibmondal/Projects.NLP/blob/master/Vietnamese_ULMFiT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
  # from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# root_dir = "/content/gdrive/My Drive/"
# base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


In [0]:
from fastai import *
from fastai.text import *

In [3]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = '/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/'
base_dir = root_dir + 'weights/VIET_NLP'

base_dir = Path(base_dir)

# check if the dirctory exists if not make it
import os
if not (os.path.exists(base_dir)):
  base_dir.mkdir(parents=True, exist_ok=True)
base_dir

path = Path('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/')
path.ls()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights')]

In [0]:
# #from fastai import *
# from fastai.text import *
# folder = 'V_S_A'

# path = Path('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data')
# dest = path/folder
# dest.mkdir(parents=True, exist_ok=True)

In [4]:
path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights')]

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [0]:
bs = 128
# bs = 48

In [0]:
data_path = path

In [9]:
data_path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights')]

In [0]:
lang = 'vi'
#lang = 'zh'
# lang = 'bn'

In [0]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [0]:
from fastai.basics import *
import re


def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        return

    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"

    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)

    with working_directory(path):
        if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 6000 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else: f.write(l)
    f.close()
    return dest

In [13]:
get_wiki(path,lang)

downloading...


unzipping...
extracting...


In [14]:
path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki-latest-pages-articles.xml'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/wikiextractor'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/log'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki')]

In [15]:
!head -n4 {path}/{name}

head: cannot open '/content/drive/My' for reading: No such file or directory
head: cannot open 'Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki' for reading: No such file or directory


In [16]:
dest = split_wiki(path,lang)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000


In [17]:
dest.ls()[:15]

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Tiếng Việt.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/California.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Thụy Điển.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Thành phố Hồ Chí Minh.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Lào Cai.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Lào.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Hoa Kỳ.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Hà Giang.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Cao Bằng.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnames

In [18]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

# data.save(f'{lang}_databunch') 
# len(data.vocab.itos),len(data.train_ds)

(60000, 17380)

In [26]:
data.show_batch()

idx,text
0,với hơn 4 triệu xxmaj việt kiều . xxmaj tiếng xxmaj việt còn là ngôn ngữ thứ hai của các dân tộc thiểu số tại xxmaj việt xxmaj nam . xxmaj mặc dù tiếng xxmaj việt có một số từ vựng vay mượn từ tiếng xxmaj hán và trước đây dùng chữ xxmaj nôm – một hệ chữ viết dựa trên chữ xxmaj hán – để viết nhưng tiếng xxmaj
1,"xxmaj trần xxmaj quang xxmaj vinh ) cũng bị bắt tại xxmaj chợ xxmaj đệm và bị giải giam tại xxmaj cà xxmaj mau cùng với xxmaj giáo sư xxmaj hồ xxmaj văn xxmaj ngà , xxmaj chủ tịch xxmaj việt xxmaj nam xxmaj độc lập xxmaj đảng . \n \n xxmaj nắm được sự mâu thuẫn này và để tranh thủ thêm đồng minh trong cuộc chiến chống"
2,"nay . xxmaj kết quả là những sợi tre được “ carbon hóa ” có khả năng chịu lửa và nhiệt độ cao – điều kiện cần thiết cho sự cháy sáng của dây tóc bóng đèn . \n \n xxmaj về sau , đến tận những năm 1950 khả năng kéo giãn của sợi carbon mới được khám phá . xxmaj người đầu tiên được cho là tạo"
3,"đo cũng như các số liệu sử dụng mà cho ra các kết quả khác nhau . xxmaj dao động từ khoảng xxunk "" m "" ( khối lượng mặt trời ) đến khoảng xxunk "" m "" , cho đến cận trên xxunk "" m "" - tương đương một nửa xxmaj thiên hà xxmaj tiên xxmaj nữ . \n \n xxmaj phần nhiều khối lượng"
4,". xxmaj một lý do khác , khí hậu mát mẻ , dễ chịu của xxmaj đà xxmaj lạt cũng tạo nhiều thuận lợi trong việc làm phim . xxmaj với những lợi thế để trở thành một phim trường thực sự , bao gồm phong cảnh thiên nhiên ưu đãi và những thắng cảnh , không gian những tòa kiến trúc , biệt thự , xxmaj đà xxmaj lạt"


In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.688340,3.545166,0.360828,57:47


In [30]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

FileNotFoundError: ignored

##Vietnamese sentiment analysis


    Data       
    Competition details       
    Top 3 f1 scores: 0.900, 0.897, 0.897        



In [25]:
%%time
data.save('vi_databunch.pkl')
# data.save(f'{lang}_databunch') 
# len(data.vocab.itos),len(data.train_ds)

CPU times: user 2.47 s, sys: 972 ms, total: 3.45 s
Wall time: 5.78 s


In [24]:
# data = load_data(path, f'{lang}_databunch', bs=bs)
data = load_data(path, 'vi_databunch', bs=bs )

FileNotFoundError: ignored

In [0]:
path.ls()

[PosixPath('/content/drive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/VIET/viwiki/viwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/content/drive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/VIET/viwiki/viwiki-latest-pages-articles.xml'),
 PosixPath('/content/drive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/VIET/viwiki/wikiextractor'),
 PosixPath('/content/drive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/VIET/viwiki/log'),
 PosixPath('/content/drive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/VIET/viwiki/viwiki')]

In [0]:
!head -n4 {path}/{name}

head: cannot open '/content/gdrive/My' for reading: No such file or directory
head: cannot open 'Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/viwiki' for reading: No such file or directory


In [0]:
dest = split_wiki(path,lang)

/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/docs already exists; not splitting


In [0]:
dest.ls()[:11]

OSError: ignored

In [0]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:454: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [0]:
data.save()

In [0]:
len(data.vocab.itos),len(data.train_ds)

(16, 1)

In [0]:
path.ls()

[PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/viwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/wikiextractor'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/log'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/text'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/viwiki'),
 PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data/viwiki/docs')]

In [0]:
data = load_data(path, f'{lang}_databunch', bs=bs)

In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: ignored

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [0]:
path.ls()

In [0]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

In [0]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(train_df.comment),'comment']='NA'
test_df.head()

In [0]:
df = pd.concat([train_df], sort=False)

In [0]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [0]:
lr = 1e-3
lr *= bs/48

In [0]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [0]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

In [0]:
!ls

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
len(data.vocab.itos), len(data.train_ds)

In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48

In [0]:
learn.unfreeze
learn.fit_one_cycle(1, lr, moms=(0.8, 0.7))

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/NLP"
base_dir = root_dir + 'fastai-v3/'


In [0]:
path = Path(base_dir + 'data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)
